In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

discussion)

(1) 사전학습 된 BERT를 대회 데이터로 추가 학습 (fine-tuning)하지 않고 임베딩 레이어와 같이 사용 가능

(2) 사전학습 되지 않은 BERT를 대회 데이터로 새로 학습하여 사용 가능

In [ ]:
train = pd.read_csv("/kaggle/input/dacon9-author-classification/train.csv", index_col = "index", encoding = 'utf-8')
test = pd.read_csv("/kaggle/input/dacon9-author-classification/test_x.csv", index_col = "index", encoding = 'utf-8')
display(train.head(),test.head())

In [1]:
# Stopword에서 작가의 어투를 잡아낼 수도 있다. 
# 일반적으로 제거하지만, 현 대회에서는 제거 시 public score 하락

# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
#              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
#              "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
#              "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
#              "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
#              "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
#              "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
#              "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
#              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
#              "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
#              "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

# train['text'] = train['text'].str.lower().apply(alpha_num)
# test['text'] = test['text'].str.lower().apply(alpha_num)

# data augmentation

In [ ]:
# !pip install nlpaug
# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc

# from nlpaug.util import Action
# from tqdm.notebook import tqdm

In [ ]:
# Synonym Augmenter (WordNet)
# from tqdm.notebook import tqdm

# aug = naw.SynonymAug(aug_src='wordnet', lang='eng')
# arr = []

# for i in tqdm(range(len(df))):
#     augmented_text = aug.augment(df.iloc[i,0])
#     arr.append(augmented_text)

# augmented_text

# Text mining

In [ ]:
# df = pd.concat([train,test])
# df

In [2]:
# 1. 단어 등록
# from keras.preprocessing.text import Tokenizer
# tk = Tokenizer()
# tk.fit_on_texts(df["text"])
# tk.word_index

In [ ]:
# 총 단어 갯수
# len(tk.word_index)

In [ ]:
# 2. text 맵핑
# all_text = tk.texts_to_sequences(df["text"])

In [ ]:
# 3. padding하기
# from keras.preprocessing.sequence import pad_sequences
# pad_sequence = pad_sequences(all_text)

In [ ]:
# train_pad = pad_sequence[:len(train)]
# test_pad = pad_sequence[len(train):]

In [ ]:
# 한 문장의 최대 단어 수
# len(train_pad[0])

# Pre train embedding - fasttext

In [ ]:
# pretrain embedding

def load_embeddings(file_name):
    embeddings = {}
    with open(file_name) as f:
        for line in f:
            values = line.rstrip().split()  # 특수문자 제거, 공백기준으로 단어가져오기
            word = values[0] # 단어1개
            vector = np.asarray(values[1:], dtype = "float32") # 숫자차원 300개 , but 200만x300개 다가져올시 많다->asarray로, float형32로 크기줄이기
            
            embeddings[word] = vector # 키word : 값value
            
    return embeddings #200만개의 딕셔녀리 값
    

# filtering
def filter_embeddings(embeddings, word_index, vocab_size , dim = 300):           # 단어, 단어종류, 단어종류개수, 단어차원
    embedding_matrix = np.zeros([vocab_size,dim]) # vocab_size x 300차원의 0행렬 생성
    
    for word,i in word_index.items():  # 딕셔너리의 key,value를 묶어서 가져옴 -> word에 the, i에 1을 각각가져옴
        vector = embeddings.get(word)     # get(word)은 word에 해당하는 딕셔너리 value를 가져옴 
        
        if vector is not None:
            embedding_matrix[i] = vector
            
    return embedding_matrix

In [ ]:
embeddings = load_embeddings("/kaggle/input/fasttext-crawl-300d-2m/crawl-300d-2M.vec")
embeddings

In [ ]:
embedding_matrix = filter_embeddings(embeddings,tk.word_index,len(tk.word_index)+1, 300)
embedding_matrix[1]

# BERT

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import sys
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
import tokenization
import warnings 
warnings.filterwarnings(action='ignore')

n_class = 5
max_len = 100
n_fold = 5
seed = 42

def bert_encode(texts, tokenizer, max_len=max_len):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


def get_model(bert_layer, max_len=max_len):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(n_class, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
trn = bert_encode(train.text.values, tokenizer, max_len=100)
tst = bert_encode(test.text.values, tokenizer, max_len=100)
y = train["author"].values

In [ ]:
model = get_model(bert_layer, max_len=max_len)
model.summary()

In [ ]:
clf = get_model(bert_layer, max_len=max_len)

clf.fit(trn,to_categorical(y), validation_split = 0.2, epochs = 3, batch_size = 16)

result = clf.predict(tst)
result

In [3]:
# Cross_validation 적용

# cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

# p_val = np.zeros((trn[0].shape[0], n_class))
# p_tst = np.zeros((tst[0].shape[0], n_class))
# for i, (i_trn, i_val) in enumerate(cv.split(trn[0], y), 1):
#     print(f'training model for CV #{i}')
#     es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
#                        verbose=1, mode='min', baseline=None, restore_best_weights=True)
    
#     clf = get_model(bert_layer, max_len=100)
#     if i == 1:
#         print(clf.summary())
        
#     clf.fit([x[i_trn] for x in trn], 
#             to_categorical(y[i_trn]),
#             validation_data=([x[i_val] for x in trn], to_categorical(y[i_val])),
#             epochs=2,
#             batch_size=16)
#     p_val[i_val, :] = clf.predict([x[i_val] for x in trn])
#     p_tst += clf.predict(tst) / 5
    

# LSTM

In [ ]:
from keras import Sequential
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from sklearn.model_selection import StratifiedKFold

rl = ReduceLROnPlateau(patience = 2, verbose = 1)
mc = ModelCheckpoint("best.h5",save_best_only = True)
es = EarlyStopping(monitor='val_loss',  patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)  # min_delta=0.001,

model = Sequential()

# 입력층
# model.add(Embedding(len(tk.word_index)+1, 40, input_length = len(train_pad[0]))) 
model.add(Embedding(len(tk.word_index)+1, 300, input_length = len(train_pad[0]), 
                    trainable = False, weights = [embedding_matrix])) # trainable = False 임베딩층안쓰고 가져온 것 쓰기 


# model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
# model.add(BatchNormalization())

model.add(Dense(64, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))

# 출력층
model.add(Dense(train["author"].nunique(),activation = "softmax")) 
model.compile(metrics = ["accuracy"], loss = "sparse_categorical_crossentropy", optimizer = "adam")

# 학습
model.fit(train_pad, train["author"], batch_size = 256, validation_split = 0.2, epochs = 100, callbacks = [es,mc,rl])

# model.load_weights("best.h5")

# result = model.predict(test_pad)

# 각 모델 Loss, accuracy

In [ ]:
# LSTM
# loss: 0.3614 - accuracy: 0.8721 - val_loss: 0.6895 - val_accuracy: 0.7615

# Bidirectional
# loss: 0.3233 - accuracy: 0.8891 - val_loss: 0.6980 - val_accuracy: 0.7622

# CNN
# loss: 2.9285 - accuracy: 0.2167 - val_loss: 2.8834 - val_accuracy: 0.1342

In [ ]:
# LSTM Cross_validaiton 적용

from sklearn.model_selection import StratifiedKFold
sfk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
result = 0

for i,j in sfk.split(train_pad,train["author"]):

    x_train = train_pad[i]
    x_valid = train_pad[j]
    
    y_train = train["author"][i]
    y_valid = train["author"][j]
        
    rl = ReduceLROnPlateau(patience = 2, verbose = 1)
    mc = ModelCheckpoint("best.h5",save_best_only = True)
    es = EarlyStopping(patience = 3, verbose = 1)

    model = Sequential()
    
    # 입력층
    model.add(Embedding(len(tk.word_index)+1, 300, input_length = len(train_pad[0]))) 

    model.add(Bidirectional(LSTM(128)))
#     model.add(LSTM(128))

    model.add(Dense(64, activation = "relu"))
    model.add(Dense(32, activation='relu'))
    
    # 출력층
    model.add(Dense(train["author"].nunique(), activation = "softmax"))

    # 편집층
    model.compile(metrics = ["acc"], loss = "sparse_categorical_crossentropy", optimizer = "adam")

    # 학습
    model.fit(train_pad, train["author"], validation_split = 0.2 ,batch_size = 256, callbacks = [es,mc,rl], epochs = 50)
    
    model.load_weights("best.h5")
    
    result += model.predict(test_pad)/5

In [ ]:
print(model.summary())

In [ ]:
sub = pd.read_csv("/kaggle/input/dacon9-author-classification/sample_submission.csv")
sub.head()

In [ ]:
sub[['0','1','2','3','4']] = pred
sub

In [ ]:
sub.to_csv("sub.csv", index=False, encoding = 'utf-8')